In [1]:
import evaluate
import numpy as np
import sys
import os
from datasets import load_from_disk, disable_caching
from sklearn.metrics import f1_score
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    set_seed,
)
sys.path.append(os.path.abspath('../../modules'))
from experiment_1.RoBERTaEntity import RoBERTaEntity

In [2]:
# import random
# seeds = [random.randint(0, 1e9) for _ in range(5)]
# seeds

In [2]:
disable_caching()

In [3]:
num_labels = 5
id2label = {
    0: "reject",
    1: "B_supplies_A",
    2: "A_supplies_B",
    3: "ambiguous",
    4: "ownership",
}
label2id = {
    "reject": 0,
    "B_supplies_A": 1,
    "A_supplies_B": 2,
    "ambiguous": 3,
    "ownership": 4,
}
metric = evaluate.load("f1")

In [4]:
model_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["__NE_FROM__", "__NE_TO__", "__NE_OTHER__"]}
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_micro = f1_score(labels, predictions, average='micro')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_classwise = f1_score(labels, predictions, average=None)

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        **{f"f1_class_{i}": score for i, score in enumerate(f1_classwise)}
    }


def model_init():
    model = RoBERTaEntity.from_pretrained(
        model_name,
        num_labels=5,
        id2label=id2label,
        label2id=label2id,
    )
    model.resize_token_embeddings(len(tokenizer))
    return model

In [6]:
ds = load_from_disk("../../datasets/ManualDataset")
ds = ds.select_columns(["masked_text", "label"])
ds = ds.rename_column("masked_text", "text")
ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2559 [00:00<?, ? examples/s]

Map:   0%|          | 0/418 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

In [8]:
def run_experiment(seed):
    set_seed(seed)
    training_args = TrainingArguments(
        seed=seed,
        data_seed=seed,
        tf32=True,
        output_dir="logs/experiment_1_roberta",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        warmup_ratio=0.1,
        load_best_model_at_end=True,
        save_total_limit=1,
        report_to=[],
        save_only_model=True,
    )
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["valid"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    test_results = trainer.predict(ds["test"])
    return test_results

In [9]:
seeds = [358951513, 487384551, 132591638, 359790194, 380316365]
all_results = []

for seed in seeds:
    results = run_experiment(seed)
    all_results.append(results)

Some weights of RoBERTaEntity were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RoBERTaEntity were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.8244170546531677, 'eval_f1_micro': 0.6913875598086124, 'eval_f1_macro': 0.5863309995645378, 'eval_f1_class_0': 0.7031700288184438, 'eval_f1_class_1': 0.2926829268292683, 'eval_f1_class_2': 0.7735191637630662, 'eval_f1_class_3': 0.6461538461538462, 'eval_f1_class_4': 0.5161290322580645, 'eval_runtime': 0.3748, 'eval_samples_per_second': 1115.207, 'eval_steps_per_second': 72.035, 'epoch': 1.0}


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.6825006604194641, 'eval_f1_micro': 0.8349282296650717, 'eval_f1_macro': 0.8226165829108402, 'eval_f1_class_0': 0.8396501457725948, 'eval_f1_class_1': 0.7076923076923077, 'eval_f1_class_2': 0.8449612403100775, 'eval_f1_class_3': 0.8571428571428571, 'eval_f1_class_4': 0.8636363636363636, 'eval_runtime': 0.3283, 'eval_samples_per_second': 1273.188, 'eval_steps_per_second': 82.239, 'epoch': 9.0}
{'loss': 0.1137, 'grad_norm': 8.375455856323242, 'learning_rate': 1.3888888888888892e-06, 'epoch': 9.38}


  0%|          | 0/27 [00:00<?, ?it/s]

{'eval_loss': 0.6699915528297424, 'eval_f1_micro': 0.8349282296650717, 'eval_f1_macro': 0.824799089455659, 'eval_f1_class_0': 0.8377581120943953, 'eval_f1_class_1': 0.7272727272727273, 'eval_f1_class_2': 0.8473282442748091, 'eval_f1_class_3': 0.848, 'eval_f1_class_4': 0.8636363636363636, 'eval_runtime': 0.3362, 'eval_samples_per_second': 1243.374, 'eval_steps_per_second': 80.314, 'epoch': 10.0}
{'train_runtime': 90.2423, 'train_samples_per_second': 283.57, 'train_steps_per_second': 17.73, 'train_loss': 0.3779454255104065, 'epoch': 10.0}


  0%|          | 0/47 [00:00<?, ?it/s]

In [10]:
# Calculate mean and std of F1 scores
metrics = ["test_f1_micro", "test_f1_macro"] + [f"test_f1_class_{i}" for i in range(5)]  # Adjust range based on num_classes

avg_results = {}
for metric in metrics:
    scores = [r[metric] for r in [x.metrics for x in all_results]]
    avg_results[metric] = {
        'mean': np.mean(scores),
        'std': np.std(scores)
    }

# Print results
for metric in metrics:
    print(f"Average {metric}: {avg_results[metric]['mean']:.4f} ± {avg_results[metric]['std']:.4f}")

Average test_f1_micro: 0.8000 ± 0.0167
Average test_f1_macro: 0.7977 ± 0.0212
Average test_f1_class_0: 0.8011 ± 0.0215
Average test_f1_class_1: 0.8147 ± 0.0250
Average test_f1_class_2: 0.8293 ± 0.0149
Average test_f1_class_3: 0.6778 ± 0.0288
Average test_f1_class_4: 0.8655 ± 0.0403
